In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import collections
import re
import pandas as pd
import numpy as np
import unicodedata
import glob
import pickle

from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [12]:
# DEFAULT_DATA_PATH = 'C:/Users/org81/Downloads/parsed/parsed/'
# DEFAULT_DATA_PATH = 'C:/Users/Rocku/Downloads/parsed/parsed/'
DEFAULT_DATA_PATH = 'Y:/SEC_Filings/'

index_file = pd.read_csv(DEFAULT_DATA_PATH + 'index_10k.csv')
index_file = index_file[index_file['f_year'] == 2008]

index_file = index_file.drop_duplicates(subset=['f_cik','f_fdate','f_ftype','acc_num'])
print(index_file.shape)
index_file.head(10)

c:\program files\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(8641, 28)


,f_cik,f_fdate,f_ftype,f_year,f_quarter,f_month,f_day,f_yymm,f_yq,f_dw,...,sic_num,irs_num,fye,form_type,sec_act,sec_file_num,film_num,former_name,number_of_filers,filer_number
129475,780127,20080104,10-K,2008,1,1,4,200801,20081,4,...,3842.0,4.11527e+08,1031.0,10-K,1934 Act,000-13907,8511112.0,BIO VASCULAR INC,1,1
129476,1039466,20080107,10-K,2008,1,1,7,200801,20081,0,...,3081.0,8.41384e+08,930.0,10-K,1934 Act,000-29621,8513749.0,SUN RIVER MINING INC,1,1
129477,1121806,20080107,10-K,2008,1,1,7,200801,20081,0,...,9995.0,5.93647e+08,930.0,10-K,1934 Act,000-32067,8515888.0,NaN,1,1
129478,1138804,20080107,10-K,2008,1,1,7,200801,20081,0,...,7372.0,4.31743e+07,NaN,10-K,1934 Act,000-51461,8515974.0,NaN,1,1
129479,1391137,20080107,10-K,2008,1,1,7,200801,20081,0,...,6035.0,0,1231.0,10-K,1934 Act,333-141572,8513153.0,NaN,1,1
129480,742550,20080107,10-K,2008,1,1,7,200801,20081,0,...,3661.0,7.3113e+08,1031.0,10-K,1934 Act,000-16231,8515264.0,XETA CORP,1,1
129481,924383,20080107,10-K,2008,1,1,7,200801,20081,0,...,3651.0,8.70362e+08,930.0,10-K,1934 Act,000-24248,8515271.0,NaN,1,1
129482,315374,20080110,10-K,2008,1,1,10,200801,20081,3,...,3823.0,3.51151e+08,1031.0,10-K,1934 Act,000-09143,8523364.0,HURCO MANUFACTURING CO INC,1,1
129483,1029800,20080111,10-K,2008,1,1,11,200801,20081,4,...,6798.0,4.2458e+07,1031.0,10-K,1934 Act,001-12803,8526406.0,HRE PROPERTIES INC,1,1
129484,103872,20080111,10-K,2008,1,1,11,200801,20081,4,...,7363.0,1.35658e+08,1028.0,10-K,1934 Act,001-09232,8525377.0,VOLT INFORMATION SCIENCES INC,1,1


In [13]:
txt_files = glob.glob(DEFAULT_DATA_PATH +'2008/*.txt')
print(len(txt_files))
txt_dict = []

for f_name in txt_files:
    temp = f_name.split('\\')[-1].split('_')[:-1]
    temp.append(f_name)
    txt_dict.append(temp)

47216


In [14]:
col = ['fdate','form_type','edgar','data','cik','acc_num','file_path']
txt_df = pd.DataFrame(txt_dict,columns=col).drop(['edgar','data'],axis=1)
txt_df.head()
txt_10k_df = txt_df[txt_df['form_type'] == '10-K']

txt_10k_df['cik'] = txt_10k_df['cik'].astype(int)
txt_10k_df['fdate'] = txt_10k_df['fdate'].astype(str)
index_file['f_cik'] = index_file['f_cik'].astype(int)
index_file['f_fdate'] = index_file['f_fdate'].astype(str)

txt_10k_df.shape

c:\program files\anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\program files\anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(8746, 5)

In [15]:
txt_merged = pd.merge(txt_10k_df, index_file, how='inner', left_on=['fdate','cik','form_type','acc_num'], right_on=['f_fdate','f_cik','form_type','acc_num'])
print(txt_merged.shape)
txt_merged.head()

(8641, 31)


,fdate,form_type,cik_x,acc_num,file_path,f_cik,f_fdate,f_ftype,f_year,f_quarter,...,sic_label,sic_num,irs_num,fye,sec_act,sec_file_num,film_num,former_name,number_of_filers,filer_number
0,20080104,10-K,780127,0000950137-08-000080,Y:/SEC_Filings/2008\20080104_10-K_edgar_data_7...,780127,20080104,10-K,2008,1,...,ORTHOPEDIC PROSTHETIC & SURGICAL APPLIANCES & ...,3842.0,4.11527e+08,1031.0,1934 Act,000-13907,8511112.0,BIO VASCULAR INC,1,1
1,20080107,10-K,1039466,0001144204-08-000872,Y:/SEC_Filings/2008\20080107_10-K_edgar_data_1...,1039466,20080107,10-K,2008,1,...,UNSUPPORTED PLASTICS FILM & SHEET,3081.0,8.41384e+08,930.0,1934 Act,000-29621,8513749.0,SUN RIVER MINING INC,1,1
2,20080107,10-K,1121806,0001010549-08-000010,Y:/SEC_Filings/2008\20080107_10-K_edgar_data_1...,1121806,20080107,10-K,2008,1,...,NaN,9995.0,5.93647e+08,930.0,1934 Act,000-32067,8515888.0,NaN,1,1
3,20080107,10-K,1138804,0000950135-08-000074,Y:/SEC_Filings/2008\20080107_10-K_edgar_data_1...,1138804,20080107,10-K,2008,1,...,SERVICES-PREPACKAGED SOFTWARE,7372.0,4.31743e+07,NaN,1934 Act,000-51461,8515974.0,NaN,1,1
4,20080107,10-K,1391137,0001193125-08-001786,Y:/SEC_Filings/2008\20080107_10-K_edgar_data_1...,1391137,20080107,10-K,2008,1,...,SAVINGS INSTITUTION FEDERALLY CHARTERED,6035.0,0,1231.0,1934 Act,333-141572,8513153.0,NaN,1,1


In [7]:
txt_example = txt_merged[0:100]

In [16]:
# def process_text(text):
#     """
#         Preprocess Text
#     """
#     text = unicodedata.normalize("NFKD", text) # Normalize
# #     text = '\n'.join(text.splitlines()) # Let python take care of unicode break lines

#     # Convert to upper
# #     text = text.upper() # Convert to upper
# #     text = re.sub(r'\n', ' ', text)
# #     text = re.sub(r'.', '.\n\n ', text)


# #     # Take care of breaklines & whitespaces combinations due to beautifulsoup parsing
# #     text = re.sub(r'[ ]+\n', '\n', text)
# #     text = re.sub(r'\n[ ]+', '\n', text)
# #     text = re.sub(r'\n+', '\n', text)
# #     text = re.sub(r',\n+', ', ', text)
# #     # To find MDA section, reformat item headers
# #     text = text.replace('\n.\n','.\n') # Move Period to beginning

# #     text = text.replace('\nI\nTEM','\nITEM')
# #     text = text.replace('\nITEM\n','\nITEM ')
# #     text = text.replace('\nITEM  ','\nITEM ')

# #     text = text.replace(':\n','.\n\n')

#     # Reformat
# #     text = text.replace('\n','\n\n') # Reformat by additional breakline
#     # remove 특수기호
#     return text

save_path = 'Y:\SEC_Filings\extracted/'
ok = []
nok = []
l = re.compile('[\W]{3,}[Ii][\s]*[Tt][\s]*[Ee][\s]*[Mm][\s]*[\n]*[1-9][AB]*[\W]*')
for ind, txt in txt_merged.iterrows():
    if ind%500 == 1:
        print(ind, txt['file_path'])
    dummy = ''
    file_name = txt['file_path'].split('\\')[-1]
    with open(txt['file_path'], "r") as f:
        dummy = f.read()
        
    splitted = l.split(dummy)
    flag =True
    for chunk in splitted:
        if (chunk.lower().strip().startswith('business') and (len(chunk) > 2000) and (flag == True)):
            with open(save_path + file_name , "w") as f:
                f.write(chunk)
            ok.append(txt['file_path'])
            flag = False
            
    if flag == True:
        nok.append(txt['file_path'])

1 Y:/SEC_Filings/2008\20080107_10-K_edgar_data_1039466_0001144204-08-000872_1.txt
501 Y:/SEC_Filings/2008\20080225_10-K_edgar_data_9984_0001193125-08-037385_1.txt
1001 Y:/SEC_Filings/2008\20080228_10-K_edgar_data_350563_0001193125-08-041825_1.txt
1501 Y:/SEC_Filings/2008\20080229_10-K_edgar_data_40211_0000950137-08-003102_1.txt
2001 Y:/SEC_Filings/2008\20080307_10-K_edgar_data_105418_0000105418-08-000013_1.txt
2501 Y:/SEC_Filings/2008\20080313_10-K_edgar_data_352363_0000950123-08-002920_1.txt
3001 Y:/SEC_Filings/2008\20080317_10-K_edgar_data_1054508_0000909654-08-000531_1.txt
3501 Y:/SEC_Filings/2008\20080318_10-K_edgar_data_1409635_0001285495-08-000031_1.txt
4001 Y:/SEC_Filings/2008\20080326_10-K_edgar_data_1392562_0001193125-08-065843_1.txt
4501 Y:/SEC_Filings/2008\20080328_10-K_edgar_data_1272597_0001104659-08-020573_1.txt
5001 Y:/SEC_Filings/2008\20080328_10-K_edgar_data_24104_0001104659-08-020842_1.txt
5501 Y:/SEC_Filings/2008\20080331_10-K_edgar_data_1372791_0000891020-08-000076_

In [10]:
with open(save_path + '2017' +'_failed.txt' , "w") as f:
    f.write(nok)

2745

In [ ]:
comp_sic_data = {
    'business_part':business_part,
    'company': company,
    'sic':sic,
    'naics':naics,
#     'snms':snms,
    'fyear':fyear,
    'exchange':ex,
    'gvkey': gvkey,
    'cik': cik,
    'ex': ex,
    'cusip': cusip,
    'gind': gind,
    'gsector': gsector
}

with open('comp_sic_merged_0806.pickle', 'wb') as f:
    pickle.dump(comp_sic_data, f)

In [ ]:
comp_sic_df = pd.DataFrame(comp_sic_data)
comp_sic_df.head()